In [40]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv

In [41]:
load_dotenv()  # take environment variables from .env.
openai_api_key = os.getenv('OPENAI_API_KEY')
milvus_api_key = os.getenv('MILVUS_API_KEY')

connection_args={
        "uri": "https://in03-5052868020ac71b.api.gcp-us-west1.zillizcloud.com",
        "user": "vaclav@pechtor.ch",
        "token": milvus_api_key,
        "secure": True
    }


In [30]:
directory = './datasets/'
docs = []

In [31]:
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    loader = TextLoader(filepath)
    doc = loader.load()
    docs.extend(doc)

In [32]:
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
docs = text_splitter.split_documents(docs)

Created a chunk of size 1162, which is longer than the specified 1024
Created a chunk of size 1239, which is longer than the specified 1024
Created a chunk of size 1081, which is longer than the specified 1024
Created a chunk of size 1078, which is longer than the specified 1024
Created a chunk of size 1173, which is longer than the specified 1024
Created a chunk of size 3424, which is longer than the specified 1024
Created a chunk of size 3257, which is longer than the specified 1024
Created a chunk of size 1985, which is longer than the specified 1024
Created a chunk of size 2477, which is longer than the specified 1024
Created a chunk of size 1078, which is longer than the specified 1024
Created a chunk of size 1173, which is longer than the specified 1024


In [33]:
from pymilvus import utility
from pymilvus import MilvusClient

client = MilvusClient(
    uri = 'https://in03-5052868020ac71b.api.gcp-us-west1.zillizcloud.com',
    token=milvus_api_key
)
client.drop_collection("Simple_DIIN_Chat")

In [34]:
embeddings = OpenAIEmbeddings()

vector_store = Milvus.from_documents(
    docs,
    embedding=embeddings,
    connection_args=connection_args,
    collection_name='Simple_DIIN_Chat'
)

In [35]:
# # Create text widget for input
# input_text = widgets.Text()

# # Create a global variable to hold the input
# query = "Warum nutzt die DIIN SAFe?"

# # Create function to update the query variable
# def update_query(change):
#     global query
#     query = change.new

# # Tell the text widget to call update_query() function when its value changes
# input_text.observe(update_query, 'value')

# # Display the text widget
# display(input_text)

In [51]:
query = "Warum nutzt die Informatik SAFe?"
print(f"Query: {query}")
docs = vector_store.similarity_search(query)

print(docs)

Query: Warum nutzt die Informatik SAFe?
[Document(page_content='Die Dienststelle Informatik stellt sich in verschiedenen Bereichen auf ein sich stark änderndes Umfeld ein. Angetrieben durch die Digitalisierung richtet sie sich nach neuen Betriebsmodellen und Technologien aus. Dies umfasst auch die Auseinandersetzung mit dem Einsatz von agilen Methoden in neuen Arbeitsumgebungen. Um die lean-agile Arbeitsweise zu verankern, macht sich die Dienststelle dem Scaled Agile Framework (SAFe) als Modell für die Organisation zu Nutze. In mehreren Agile Teams, die in einem Agile Release Train organisiert sind, arbeitet die Dienststelle an den künftigen Lösungen für ihre Kundinnen und Kunden. \nSAFe\xa0stellt eine Reihe von Unternehmens- und Workflow-Mustern zur Implementierung von agilen Praktiken im gesamten Unternehmen bereit. Die Dienststelle Informatik orientiert sich unter anderem an strukturierten Leitlinien zu Rollen und Zuständigkeiten sowie zur Planung und Verwaltung von Aufgaben, die es

In [73]:
from langchain.prompts import PromptTemplate
language = "Swiss German Dialect"
prompt_template = f"""You are an assistant that answers questions about the Kanton Luzern,
 based on given information. Only use the information that was provided below.
 Use the following pieces of context to answer the question at the end.
 If you don't know the answer, just say that you don't know, don't try to make up an answer.

{{context}}

Question: {{question}}
Answer in {language}:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [83]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-0613',temperature=0.0)
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 retriever=vector_store.as_retriever(),
                                 chain_type='stuff',
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs)
result = qa({"query": query})

In [84]:
print(f'Query: {query}')
print(f'Result: {result}')

Query: Warum nutzt die Informatik SAFe?
Result: {'query': 'Warum nutzt die Informatik SAFe?', 'result': "Die Dienststelle Informatik nutzt SAFe (Scaled Agile Framework) als Modell für d'Organisation, um d'lean-agile Arbeitswiis z'verankere. Mit SAFe chönd sie sich besser uf neui Betriebsmodelle und Technologie iistelle und agili Methode in neui Arbeitsumgebige iiführe. SAFe bietet au strukturierti Leitlinie zu Rolle und Zuständigkeite sowie zur Planig und Verwaltig vo Aufgabe, was der Dienststelle hilft, de Nutze für ihre Chunde ständig z'optimiere. D'Ziel isch, qualitativ hochstehendi Leistige so schnell wie möglich z'erbringe.", 'source_documents': [Document(page_content='Die Dienststelle Informatik stellt sich in verschiedenen Bereichen auf ein sich stark änderndes Umfeld ein. Angetrieben durch die Digitalisierung richtet sie sich nach neuen Betriebsmodellen und Technologien aus. Dies umfasst auch die Auseinandersetzung mit dem Einsatz von agilen Methoden in neuen Arbeitsumgebungen.